## Looking Ahead: LangChain and LangSmith for Future Classes

Now that you understand our agent architecture with custom tools, FAISS RAG, and Gemini integration, let's explore how industry-standard frameworks like **LangChain** and **LangSmith** could enhance and streamline this workflow in future iterations of this course.

### What is LangChain?

**LangChain** is an open-source framework for building applications powered by Large Language Models (LLMs). It provides abstractions and tools for:
- **Chains**: Sequential operations that process inputs through multiple steps
- **Agents**: Autonomous systems that decide which tools to use
- **Tools**: Reusable functions that agents can call
- **Memory**: Persistent context across multiple interactions
- **Retrievers**: Built-in RAG and vector database integrations

**Official website**: https://www.langchain.com/  
**Documentation**: https://python.langchain.com/docs/

### Why LangChain Would Be Useful for Our Agent

#### 1. Built-in RAG Components

**What we did manually:**
```python
# Our custom implementation
retriever = build_retriever(db)
rag_contexts = retriever.search(search_query, k=3)
```

**With LangChain:**
```python
from langchain.vectorstores import FAISS
from langchain.embeddings import GoogleGenerativeAIEmbeddings

# More powerful, standardized
embeddings = GoogleGenerativeAIEmbeddings(model="text-embedding-004")
vectorstore = FAISS.from_documents(documents, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
```

**Benefits:**
- **Pre-built integrations** with 50+ embedding models and vector stores
- **Standardized interfaces** that work across different providers
- **Advanced features** like MMR (Maximal Marginal Relevance) for diversity
- **Easy switching** between FAISS, Pinecone, Weaviate, Chroma, etc.

#### 2. Agent Frameworks

**What we did manually:**
```python
# Our custom agent orchestration
def run_release_readiness_agent(context, db):
    brief = fetch_feature_brief(context.feature_slug)
    launch_window = fetch_launch_window(context.feature_slug)
    rag_contexts = retriever.search(...)
    gemini_insight = _generate_gemini_insight(...)
    # ... manual orchestration
```

**With LangChain:**
```python
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain.tools import tool

@tool
def fetch_feature_brief(feature_slug: str) -> dict:
    """Fetch feature brief for a given feature slug."""
    return {...}

@tool  
def retrieve_docs(query: str) -> list[str]:
    """Retrieve relevant documentation using RAG."""
    return vectorstore.similarity_search(query, k=3)

# Agent decides which tools to use and when
agent = create_openai_functions_agent(llm, tools=[fetch_feature_brief, retrieve_docs])
executor = AgentExecutor(agent=agent, tools=tools)
result = executor.invoke({"input": "Analyze release readiness for curriculum-pathways"})
```

**Benefits:**
- **Autonomous decision-making**: Agent chooses which tools to call
- **Automatic retries**: Built-in error handling and retry logic
- **Streaming**: Stream responses in real-time
- **Multiple agent types**: ReAct, OpenAI Functions, Structured Chat, etc.

#### 3. Prompt Templates and Chains

**What we did manually:**
```python
prompt = f"""You are a release readiness advisor...
Context:
{full_context}
User's Launch Date: {context.launch_date}
..."""
response = model.generate_content(prompt)
```

**With LangChain:**
```python
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a release readiness advisor helping teams prepare for launches."),
    ("human", "Based on this context: {context}\n\nProvide analysis for launch on {launch_date}")
])

chain = LLMChain(llm=llm, prompt=prompt)
result = chain.invoke({"context": full_context, "launch_date": context.launch_date})
```

**Benefits:**
- **Reusable templates**: Define prompts once, use everywhere
- **Validation**: Type-checked inputs and outputs
- **Composition**: Chain multiple LLM calls together
- **Few-shot examples**: Easily add example inputs/outputs

#### 4. Memory Systems

**What we currently have:**
```python
# Static: Each agent run is independent
agent_run = AgentRun(...)
db.add(agent_run)
```

**With LangChain:**
```python
from langchain.memory import ConversationBufferMemory

# Dynamic: Agent remembers previous interactions
memory = ConversationBufferMemory()
chain = ConversationChain(llm=llm, memory=memory)

# First call
chain.invoke("What's the launch date for curriculum-pathways?")

# Second call remembers context
chain.invoke("What are the risks?")  # Knows we're still talking about curriculum-pathways
```

**Benefits:**
- **Conversation tracking**: Multi-turn dialogues
- **Context window management**: Automatic summarization
- **Multiple memory types**: Buffer, summary, entity, vector store-backed

### What is LangSmith?

**LangSmith** is a platform for **debugging, testing, evaluating, and monitoring** LLM applications. Think of it as the observability layer for AI agents.

**Official website**: https://www.langchain.com/langsmith  
**Documentation**: https://docs.smith.langchain.com/

### Why LangSmith Would Be Critical for Production

#### 1. Observability and Debugging

**What we currently have:**
```python
# Limited: Just tool call traces in response
tool_calls = [
    AgentToolCall(tool="fetch_feature_brief", ...),
    AgentToolCall(tool="rag_retrieval", ...),
]
```

**With LangSmith:**
- **Full trace visualization**: See every step of agent execution
- **Input/output inspection**: Examine prompts and responses
- **Latency breakdown**: Identify slow components
- **Token usage tracking**: Monitor costs per run
- **Error tracking**: Automatic error capture with context

**Example trace:**
```
Agent Run (2.3s, $0.02)
├─ fetch_feature_brief (0.1s)
│  └─ Input: {"feature_slug": "curriculum-pathways"}
│  └─ Output: {"name": "Curriculum Pathways", ...}
├─ RAG retrieval (0.4s)
│  ├─ Embedding (0.1s, 5 tokens)
│  └─ FAISS search (0.3s)
│  └─ Output: [3 documents]
├─ Gemini insight (1.8s, $0.02)
│  ├─ Prompt (234 tokens)
│  ├─ Response (156 tokens)
│  └─ Output: "Strategic launch assessment..."
└─ Result assembled (0.0s)
```

#### 2. Dataset Management and Testing

**What we currently lack:**
```python
# Manual testing, no test suite
# Have to run agent manually to verify behavior
```

**With LangSmith:**
```python
# Create datasets for evaluation
dataset = client.create_dataset("release_readiness_tests")

# Add test cases
client.create_examples(
    dataset_id=dataset.id,
    inputs=[
        {"feature_slug": "curriculum-pathways", "launch_date": "2025-03-01"},
        {"feature_slug": "ai-code-review", "launch_date": "2025-04-15"},
    ],
    outputs=[
        {"expected_recommendations": ["Confirm launch communications", ...]},
        {"expected_recommendations": ["Validate operational readiness", ...]},
    ]
)

# Run evaluation
results = client.run_on_dataset(
    dataset_name="release_readiness_tests",
    llm_or_chain=agent_chain,
    evaluation=custom_evaluator
)
```

**Benefits:**
- **Regression testing**: Ensure changes don't break existing behavior
- **A/B testing**: Compare different prompts or models
- **Ground truth comparison**: Measure accuracy against expected outputs

#### 3. Prompt Iteration and Optimization

**What we currently do:**
```python
# Manual: Edit prompt string, redeploy, test manually
prompt = f"You are a release readiness advisor..."
```

**With LangSmith:**
- **Prompt playground**: Test prompts interactively
- **Version control**: Track prompt changes over time
- **Comparison view**: See outputs side-by-side
- **Automatic optimization**: Find best-performing prompt variants

#### 4. Production Monitoring

**What we currently have:**
```python
# Basic: Save runs to database
agent_run = AgentRun(...)
db.add(agent_run)
```

**With LangSmith:**
- **Real-time dashboards**: Monitor agent performance live
- **Alerting**: Get notified of errors or anomalies
- **Cost tracking**: See token usage and costs per endpoint
- **User feedback**: Collect thumbs up/down from users
- **Automated analysis**: Identify patterns in failures

### Migration Path: From Our Implementation to LangChain

If we were to adopt LangChain in future classes, here's how we'd migrate:

#### Phase 1: RAG Layer (Easiest)
```python
# Replace app/services/rag.py with LangChain vectorstore
from langchain.vectorstores import FAISS
from langchain.embeddings import GoogleGenerativeAIEmbeddings

vectorstore = FAISS.from_documents(chunks, GoogleGenerativeAIEmbeddings())
```

#### Phase 2: Tool Definitions
```python
# Convert app/services/agent_tools.py to LangChain tools
from langchain.tools import tool

@tool
def fetch_feature_brief(feature_slug: str) -> FeatureBrief:
    """Fetch feature brief for release planning."""
    return FeatureBrief(...)
```

#### Phase 3: Agent Logic
```python
# Replace app/services/agent.py with LangChain agent
from langchain.agents import create_openai_functions_agent

agent = create_openai_functions_agent(
    llm=ChatGoogleGenerativeAI(model="gemini-2.0-flash"),
    tools=[fetch_feature_brief, fetch_launch_window, retrieve_docs],
    prompt=prompt_template
)
```

#### Phase 4: Add LangSmith
```python
import os
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "your-key"

# That's it! All runs now traced automatically
```

### When to Use LangChain vs Custom Implementation

**Use custom implementation (like ours) when:**
- ✅ Learning fundamentals (educational context)
- ✅ Full control over every detail
- ✅ Minimal dependencies
- ✅ Simple, deterministic workflows
- ✅ Offline operation required

**Use LangChain when:**
- ✅ Building production systems
- ✅ Need to iterate quickly on prompts
- ✅ Want built-in observability
- ✅ Need to support multiple LLM providers
- ✅ Complex agent orchestration
- ✅ Team collaboration on prompts

### Hands-on Exercise for Future Classes

**Goal**: Convert one part of our agent to use LangChain

1. **Install LangChain**: `pip install langchain langchain-google-genai`
2. **Replace RAG**: Use `langchain.vectorstores.FAISS` instead of our custom `rag.py`
3. **Add tracing**: Enable LangSmith to visualize agent execution
4. **Compare**: Measure performance, code complexity, and developer experience

### Resources for Deeper Learning

**LangChain:**
- 📘 **Official Docs**: https://python.langchain.com/docs/
- 🎓 **Tutorials**: https://python.langchain.com/docs/tutorials/
- 📦 **Agent Templates**: https://python.langchain.com/docs/modules/agents/
- 💬 **Community**: Discord and GitHub discussions

**LangSmith:**
- 📘 **Docs**: https://docs.smith.langchain.com/
- 🎥 **Demo Videos**: https://www.langchain.com/langsmith
- 📊 **Evaluation Guide**: https://docs.smith.langchain.com/evaluation
- 🔍 **Tracing Guide**: https://docs.smith.langchain.com/tracing

### Key Takeaways

1. **Our implementation taught you the fundamentals** - You now understand:
   - How embeddings work
   - How FAISS indexes documents
   - How RAG retrieves context
   - How agents orchestrate tools
   - How to structure AI applications

2. **LangChain provides production-ready abstractions** - It handles:
   - Provider integrations (Gemini, OpenAI, Anthropic, etc.)
   - Prompt management and versioning
   - Agent frameworks and tooling
   - Error handling and retries

3. **LangSmith enables production monitoring** - Essential for:
   - Debugging complex agent behavior
   - Evaluating prompt changes
   - Tracking costs and latency
   - Collecting user feedback

4. **The choice depends on your context**:
   - **Learning/Teaching**: Custom implementation (better understanding)
   - **Production**: LangChain + LangSmith (faster iteration, better tooling)
   - **Hybrid**: Start custom, migrate to LangChain as complexity grows

By understanding both approaches, you're equipped to:
- Build AI agents from first principles
- Adopt industry-standard frameworks when appropriate
- Make informed architectural decisions
- Debug and optimize agent behavior
- Scale from prototype to production


## Homework / extensions

The current implementation already includes what were previously extension challenges:
- ✅ **Gemini AI integration** for generating strategic insights
- ✅ **FAISS RAG** for retrieving relevant documentation
- ✅ **Database persistence** for auditing agent runs
- ✅ **Priority levels** for recommendations
- ✅ **Frontend integration** with the AgentPanel component

### New Extension Ideas

1. **Add more sophisticated prompting**
   - Implement few-shot examples in Gemini prompts
   - Add chain-of-thought reasoning
   - *Teaching moment*: Prompt engineering for production systems

2. **Implement agent memory**
   - Use past runs to inform new recommendations
   - Track feature launch patterns over time
   - *Teaching moment*: Stateful agents and learning from history

3. **Add real-time streaming**
   - Stream Gemini responses to the frontend
   - Show progressive tool call results
   - *Teaching moment*: Server-sent events and async patterns

4. **Create an agent evaluation framework**
   - Define test cases with expected recommendations
   - Measure recommendation quality over time
   - *Teaching moment*: Evaluating AI systems

5. **Add multi-agent orchestration**
   - Create specialized sub-agents for different concerns
   - Implement agent-to-agent communication
   - *Teaching moment*: Multi-agent architectures

### Advanced Challenges

- **Implement semantic caching** to avoid repeated RAG queries
- **Add confidence scores** to recommendations
- **Create a feedback loop** where users rate recommendations
- **Implement A/B testing** for different prompting strategies


# Lab 06 · Embeddings and FAISS

*This lab notebook provides guided steps. All commands are intended for local execution.*

## Objectives
- Document chunking routines are introduced.
- The "text-embedding-004" vectors are stored in a FAISS index.
- A search endpoint returns top results with scores.

## What will be learned
- Document preprocessing for embeddings is rehearsed.
- FAISS index persistence is described.
- Vector search endpoints are surfaced.

## Prerequisites & install
The following commands are intended for local execution.

```bash
cd ai-web/backend
. .venv/bin/activate
pip install faiss-cpu google-generativeai numpy
```

## Step-by-step tasks
### Step 1: Chunking utility
A chunking helper is added so documents are segmented.

In [ ]:
from pathlib import Path
vector_path = Path("ai-web/backend/app/vector.py")
vector_path.write_text('''import json
import os
from pathlib import Path
from typing import List, Tuple

from google import genai
import numpy as np
import faiss


DATA_DIR = Path(__file__).resolve().parent / "data"
DATA_DIR.mkdir(parents=True, exist_ok=True)
INDEX_FILE = DATA_DIR / "embeddings.index"
META_FILE = DATA_DIR / "metadata.json"


def _client() -> genai.Client:
  api_key = os.environ.get('GEMINI_API_KEY', '')
  if not api_key:
    raise RuntimeError('A backend API key is required for embeddings.')
  return genai.Client(api_key=api_key)


def _text_content(text: str) -> dict:
  return {"parts": [{"text": text}]}


def chunk_text(text: str, size: int = 400) -> List[str]:
  return [text[i:i + size] for i in range(0, len(text), size) if text[i:i + size].strip()]


def embed_chunks(chunks: List[str]) -> np.ndarray:
  client = _client()
  response = client.models.embed_content(
      model='text-embedding-004',
      contents=[_text_content(chunk) for chunk in chunks],
  )
  embeddings = response.embeddings or []
  if not embeddings:
    raise RuntimeError('No embeddings were returned from the Gemini API.')
  return np.array([item.values for item in embeddings], dtype=np.float32)


def save_index(chunks: List[str], vectors: np.ndarray) -> None:
  index = faiss.IndexFlatIP(vectors.shape[1])
  faiss.normalize_L2(vectors)
  index.add(vectors)
  faiss.write_index(index, str(INDEX_FILE))
  META_FILE.write_text(json.dumps({"chunks": chunks}))


def load_index() -> Tuple[faiss.Index, List[str]]:
  index = faiss.read_index(str(INDEX_FILE))
  chunks = json.loads(META_FILE.read_text())["chunks"]
  return index, chunks


def search(query: str, top_k: int = 3) -> List[Tuple[str, float]]:
  index, chunks = load_index()
  query_vec = embed_chunks([query])
  faiss.normalize_L2(query_vec)
  scores, neighbors = index.search(query_vec, top_k)
  return [(chunks[i], float(scores[0][pos])) for pos, i in enumerate(neighbors[0])]
''')
print("Vector helper was written.")

### Step 2: Index builder cell
An index is created from a small sample document.

In [ ]:
import sys
from pathlib import Path

sys.path.append(str(Path('ai-web/backend')))
from app.vector import chunk_text, embed_chunks, save_index

sample_text = """This course demonstrates AI in web programming.
The backend relies on FastAPI and Gemini proxies.
Vector search provides relevant snippets.
"""
chunks = chunk_text(sample_text)
vectors = embed_chunks(chunks)
save_index(chunks, vectors)
print('Index was generated with', len(chunks), 'chunks.')

### Step 3: Search endpoint
A FastAPI endpoint is published for vector search.

In [ ]:
from pathlib import Path
main_path = Path("ai-web/backend/app/main.py")
text = main_path.read_text()
if "search_endpoint" not in text:
    addition = '''
from typing import Optional
from .vector import search


@app.get("/api/search")
def search_endpoint(q: str, k: Optional[int] = 3):
    results = search(q, int(k))
    return {"results": [{"text": text, "score": score} for text, score in results]}
'''
    main_path.write_text(text.rstrip() + "
" + addition)
    print("Search endpoint was appended.")
else:
    print("Search endpoint already present.")

## Validation / acceptance checks
```bash
# locally
curl 'http://localhost:8000/api/search?q=fastapi&k=2'
```
- A JSON response containing scored chunks is observed.
- React development mode shows the described UI state without console errors.

## Homework / extensions
- Periodic index rebuild strategies are evaluated for large document sets.
- Client-side rendering of search results is explored.